## How to use Mathematical Program, and use it to formulate optimizations for robots

One area of tools offered in pydrake is the MathematicalProgram interface.  MathematicalProgram is a class that abstracts many different useful types of optimization solvers.  This makes it so from one interface, you can access many different solvers.  The MathematicalProgram component of Drake is comparable to JuMP, in the Julia ecosystem. To get a concise overview of which solvers are supported for which different types of optimization problems, check out [this chart](http://drake.mit.edu/doxygen_cxx/group__solvers.html).  

As is the case with hot-off-the-presses code, there is not yet a plethora of documentation, but here we're going to try to provide you with enough sample code to help get you started. 

In addition to the code snippets below, these two tips are also very useful:

- Once you construct a MathematicalProgram object, i.e. `mp = MathematicalProgram()`, the tab completion in your jupyter notebook can be very helpful.  

 --> For example, let's say you want to know if a MathematicalProgram can print how many decision variables currently exist.  Tab completing on `mp.` and scrolling through, you'll find `num_vars`.  Indeed `mp.num_vars()` will do the trick.
 
 --> Want to know which solver MP is currently using under the hood for a particular problem instance?
 
```python
res = Solve(mp) # must first solve the program (which forces a solver to be chosen)
solver = res.get_solver_id()
solver.name() # name will tab complete after creating a solver object
```
- An additional resource for how to use MathematicalProgram is the tests written for it. There are a significant amount of tests for MathematicalProgram, written in C++.  See [here](https://github.com/RobotLocomotion/drake/blob/master/solvers/test/mathematical_program_test.cc) but also other tests in that folder.  Note however that not all C++ features have pydrake bindings -- for those familiar with pybind, the bindings for MathematicalProgram are generated [here](https://github.com/RobotLocomotion/drake/blob/master/bindings/pydrake/solvers/mathematicalprogram_py.cc), and are demonstrated in numerous tests [here](https://github.com/RobotLocomotion/drake/tree/master/bindings/pydrake/solvers/test).

### Okay, but how do I actually do an optimization problem?  

How do we translate something written on the board as a linear program, and write it down in code?

Here is a very simple example of an LP:

\begin{align*}
        \min_{x} \ \ \ x \\
        s.t. \ \ \  & x >= 1 \\
\end{align*}

And the corresponding Mathematical Program code is below.

In [1]:
from pydrake.all import MathematicalProgram, SolverType, Solve, SolverOptions
from pydrake.solvers.ipopt import IpoptSolver
import numpy as np
import math

In [2]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(x[0]*1.0)
mp.AddLinearConstraint(x[0] >= 1)
res = Solve(mp)
print("success:", res.is_success())
print(res.get_solver_id().name())
print(res.GetSolution(x))
print("solution result:", res.get_solution_result())

success: True
SNOPT/f2c
[1.]
solution result: SolutionResult.kSolutionFound


In [3]:
# solve the same program as above but with the IPOPT solver
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(x[0]*1.0)
mp.AddLinearConstraint(x[0] >= 1)
solver = IpoptSolver()
res = solver.Solve(mp, None, None)
print("success:", res.is_success())
print(res.get_solver_id().name())
print(res.GetSolution(x))
print("solution result:", res.get_solution_result())

success: True
IPOPT
[1.]
solution result: SolutionResult.kSolutionFound


Note that written down slightly incorrectly, you will not get the answer you were looking for.  What is wrong about the two examples below?

In [4]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(x[0]*1.0)
res = Solve(mp)
print(res.is_success())
print(res.GetSolution(x))

False
[nan]


In [5]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(x[0]*1.0)
mp.AddLinearConstraint(x[0] <= 1)
res = Solve(mp)
print(res.is_success())
print(res.GetSolution(x))

False
[-6.17673396e+14]


Here's a slightly more complicated example, this one solves a problem that may look familiar to you.

This is just one example of how, even though Linear Programs can only handle linear objectives and constraints, you can use them to sample over arbitrarily complex functions, and the samples of those functions can still be just linear constraints / objectives.

In [6]:
mp = MathematicalProgram()
alpha = mp.NewContinuousVariables(1, "alpha")
mp.AddLinearCost(alpha[0]*1.0)
for xi in np.arange(-5*np.pi, 5*np.pi+np.pi/8, np.pi/8):
    mp.AddLinearConstraint(alpha[0] - math.cos(xi)**2 + math.sin(xi) >= 0)
    
res = Solve(mp)
print(res.is_success())
print(res.GetSolution(alpha))

True
[1.23623682]


Note the MathematicalProgram is formulated in terms of "costs", and will minimize the objective function's costs when calling `Solve()`.  How can we maximize functions? Just add a negative sign:

In [ ]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddLinearCost(-x[0]*1.0)
mp.AddLinearConstraint(x[0] <= 4)
res = Solve(mp)
print(res.is_success())
print(res.GetSolution(x))

Now how about if we want to go outside the realm of Linear Programs?  What if we want to do a Quadratic Program?  Recall that the only difference between a quadratic program and a Linear Program is that QPs now allow a quadratic cost, but still only linear objectives.

In [ ]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(1, "x")
mp.AddQuadraticCost((x[0]-3)**2)
res = Solve(mp)
print("success:", res.is_success())
print(res.get_solver_id().name())
print(res.GetSolution(x))
print("solution result:", res.get_solution_result())

Note that as above, a QP can be well formulated even without any constraints.  (LPs will have unbounded objectives without constraints.)

Note that there is no `QuadraticConstraint` in MathematicalProgram.  Why not?  (What class of problem is a QuadraticConstraint?)

But actually although there is no specific function call for it, MathematicalProgram can generally handle a quadratic constraint, and many other different types of constraints, through `AddConstraint`, where inside the argument to the function is a symbolic formula of type `==`, `>=`, or `<=`.  This opens up MathematicalProgram to solve general nonlinear optimization problems.

In [ ]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(2, "x")
mp.AddConstraint((x[0]**2 + x[1]**2) == 1.0)
mp.AddLinearCost(x.sum())

# note if you set the initial guess to be zero,
# then SNOPT fails to find a solution, if you set it to be
# -0.5 it gives the correct solution
mp.SetInitialGuessForAllVariables(np.array([-0.5,-0.5]))

# this will give the incorrect solution. Remember this is a non-convex optimization
# and SNOPT is a local solver
# mp.SetInitialGuessForAllVariables(np.array([0.5,0.5]))

res = Solve(mp)
print("success:", res.is_success())
print(res.get_solver_id().name())
print(res.GetSolution(x))
print("solution result:", res.get_solution_result())
solver_id = res.get_solver_id()

Alternatively you can even use many numpy operations, including `dot`:

In [ ]:
mp = MathematicalProgram()
x = mp.NewContinuousVariables(2, "x")
mp.AddConstraint(x.dot(x) == 1.0)
mp.AddLinearCost(x.sum())
mp.SetInitialGuessForAllVariables(np.array([-0.5, -0.5]))
res = Solve(mp)
print(res.is_success())
print(res.GetSolution(x))
print(res.get_solution_result())

The above type of constraint will be very useful for our trajectory optimizations, which are in general nonlinear for sufficiently complex problems.

Note below that for the Mixed-Integer class of problems, the two solvers supported by Drake (as you can see from the chart linked above) are both proprietary solvers and not shipped with the docker image.  You may want to delve into this class of problems for final projects, however, and we can help you get access to these solvers if needed.  Gurobi is very powerful and availble for free for academic use, although will not work easily in a docker image and so we'd suggest a native Drake installation.

In [ ]:
# This needs a commercial solver to work...
# mp = MathematicalProgram()
# x = mp.NewContinuousVariables(1, "x")
# b = mp.NewBinaryVariables(1, "b")
# mp.AddLinearConstraint(x[0] <= 1)
# mp.Solve()
# print mp.GetSolution(x)

## Custom costs and constraints
You can add your own costs/constraints that don't fit into one of the specified forms

In [ ]:
def my_cost_func(x):
    return np.dot(x,x)

mp = MathematicalProgram()
x = mp.NewContinuousVariables(2, "x")
mp.AddCost(my_cost_func, x) # there is a similar AddConstraint function as well
mp.AddConstraint(x[1] >= 1.0)

res = Solve(mp)
print("success:", res.is_success())
print("x:", res.GetSolution(x))
print(res.get_solution_result())
print("cost:", res.get_optimal_cost())